"some_name".txt: Usually **ASCII** or plain text.

"some_name"**-0**.txt: Encoded in **ISO-8859-1** (Latin-1).

"some_name"**-8**.txt: Encoded in **UTF-8**.

https://www.gutenberg.org/help/file_formats.html#unicodeutf-8-utf-16-utf-32-character-set


In [1]:
import glob
import os
import io
import pandas as pd
import chardet
from pandas import read_csv

#Settings 
GLOBAL_PATH = None

# CSV containing pg catalogue and year estimates
pg_catalogue_with_years_loc = None

NameError: name 'TODO' is not defined

In [2]:
gutenberg_catalog = read_csv(pg_catalogue_with_years_loc, dtype=str)

In [3]:
gutenberg_catalog.loc[:, "year_majority_vote"] = gutenberg_catalog.apply(
    lambda row: int(float(row["year_majority_vote"])) if not pd.isna(row["year_majority_vote"]) else row["year_majority_vote"], 
    axis=1
)
gutenberg_catalog.loc[:, "first_author_birth_year"] = gutenberg_catalog.apply(
    lambda row: int(float(row["first_author_birth_year"])) if not pd.isna(row["first_author_birth_year"]) else row["first_author_birth_year"], 
    axis=1
)
gutenberg_catalog.loc[:, "first_author_death_year"] = gutenberg_catalog.apply(
    lambda row: int(float(row["first_author_death_year"])) if not pd.isna(row["first_author_death_year"]) else row["first_author_death_year"], 
    axis=1
)

In [4]:
gutenberg_catalog[:-1]

,id,title,authors,first_author_birth_year,first_author_death_year,translators,type,subjects,languages,formats,downloads,bookshelves,copyright,year_google_api,year_google_html,year_loc,year_open_library,year_wikipedia,year_title,year_majority_vote
0,1,The Declaration of Independence of the United ...,"Jefferson, Thomas",1743.0,1826.0,NaN,Text,"United States -- History -- Revolution, 1775-1...",en,"text/html, application/epub+zip, application/x...",2249,"American Revolutionary War, United States Law,...",False,NaN,NaN,NaN,1776,1776,NaN,1776.0
1,2,The United States Bill of Rights: The Ten Orig...,United States,NaN,NaN,NaN,Text,"Civil rights -- United States -- Sources, Unit...",en,"text/html, application/epub+zip, application/x...",2626,"American Revolutionary War, United States Law,...",False,NaN,NaN,NaN,1787,NaN,NaN,1787.0
2,3,John F. Kennedy's Inaugural Address,"Kennedy, John F. (John Fitzgerald)",1917.0,1963.0,NaN,Text,Presidents -- United States -- Inaugural addre...,en,"text/html, application/epub+zip, application/x...",974,NaN,False,NaN,NaN,NaN,1962,1961,NaN,1961.0
3,4,Lincoln's Gettysburg Address: Given November 1...,"Lincoln, Abraham",1809.0,1865.0,NaN,Text,Consecration of cemeteries -- Pennsylvania -- ...,en,"text/html, application/epub+zip, application/x...",969,US Civil War,False,NaN,NaN,NaN,NaN,NaN,1863,1863.0
4,5,The United States Constitution,United States,NaN,NaN,NaN,Text,United States -- Politics and government -- 17...,en,"text/html, text/html; charset=us-ascii, applic...",815,"American Revolutionary War, United States, Pol...",False,NaN,1787,NaN,2011,1789,NaN,1789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73564,73646,Tarinoita,"Krylov, I. A.",NaN,NaN,"Putro, Sakari",Text,NaN,fi,text/plain; charset=us-ascii,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73565,73647,The psychology of speculation : $b The human e...,"Harper, Henry Howard",1871.0,1953.0,NaN,Text,NaN,en,"text/html, text/plain; charset=us-ascii",0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73566,73648,"After the stock market crash of November, 1929...","Harper, Henry Howard",1871.0,1953.0,NaN,Text,NaN,en,"text/html, text/plain; charset=us-ascii",0,NaN,False,NaN,NaN,NaN,NaN,NaN,1929,1929.0
73567,73649,"Ye butcher, ye baker, ye candlestick-maker : $...","Seaver, Robert",NaN,NaN,NaN,Text,NaN,en,"text/html, text/plain; charset=us-ascii",0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tmp = gutenberg_catalog.loc[pd.isna(gutenberg_catalog["translators"])]
tmp = tmp.loc[(tmp["languages"] == "en")]
tmp = tmp.loc[tmp["type"]=="Text"]
tmp = tmp.loc[tmp["copyright"]=="False"]
tmp = tmp.loc[(tmp["authors"].notnull()) & (tmp['authors'] != 'Unknown')]
tmp = tmp.loc[tmp["first_author_birth_year"].notnull()]
tmp = tmp.loc[(tmp["year_title"].notnull()) | (tmp["year_wikipedia"].notnull()) | (tmp["year_open_library"].notnull())]
tmp = tmp.loc[tmp["year_majority_vote"]>tmp["first_author_birth_year"]]
tmp = tmp.loc[(pd.isna(tmp["first_author_death_year"]) & (tmp["year_majority_vote"]- tmp["first_author_birth_year"] <= 100)) | (tmp["year_majority_vote"]<= tmp["first_author_death_year"])]
tmp = tmp.loc[(tmp["year_majority_vote"]>= 1750) & (tmp["year_majority_vote"]<=2000)]
ids = tmp['id'].tolist()
#available as .txt in PG
years = tmp['year_majority_vote'].tolist()
print(len(ids))
filtered_df = tmp[tmp['year_majority_vote'] == 1998]

# Display the filtered DataFrame
print(filtered_df)

25136
Empty DataFrame
Columns: [id, title, authors, first_author_birth_year, first_author_death_year, translators, type, subjects, languages, formats, downloads, bookshelves, copyright, year_google_api, year_google_html, year_loc, year_open_library, year_wikipedia, year_title, year_majority_vote]
Index: []


In [6]:
def build_file_path(id):
    if int(id) < 10:
        return f"0/{id}/"
    path = ""
    nr_list = [n for n in str(id)][:-1]
    for n in nr_list:
        path += f"{n}/"
    path += str(id)
    path += "/"

    return path

In [7]:
print(build_file_path(8))

0/8/


In [8]:
def get_text_files(path):
    res = glob.glob(f'{GLOBAL_PATH}{path}/*.txt')
    #if f"/content/drive/MyDrive/Master Thesis/{path}/{path.split('/')[-1]}.txt" in res:
    #  return f"/content/drive/MyDrive/Master Thesis/{path}/{path.split('/')[-1]}.txt"
    #else:
    #  raise Exception("No asci encoding")
    return res

In [9]:
get_text_files(build_file_path(50426))

['/home/ljf/mirror/5/0/4/2/50426/50426-0.txt']

In [10]:
def build_encoding_matrix(ids):
    matrix = {"ids":ids, "ascii": [], "utf": [], "ISO-8859-1": [], "readme": []}
    files_missing = []
    for id in ids:
        path = build_file_path(id)
        txts = get_text_files(path)
        
        txtFound = False
        #check for ASCII encoding
        if f"{GLOBAL_PATH}{path}{path.split('/')[-2]}.txt" in txts:
            matrix["ascii"].append(True)
            txtFound = True
        else:
            matrix["ascii"].append(False)
    
        #check for UTF-8 Encoding
        if f"{GLOBAL_PATH}{path}{path.split('/')[-2]}-8.txt" in txts:
            matrix["utf"].append(True)
            txtFound = True
        else:
            matrix["utf"].append(False)
          
        # Check for ISO-8859-1 encoding
        if f"{GLOBAL_PATH}{path}{path.split('/')[-2]}-0.txt" in txts:
            matrix["ISO-8859-1"].append(True)
            txtFound = True
        else:
            matrix["ISO-8859-1"].append(False)
            
        # Check for readme
        if f"{GLOBAL_PATH}{path}{path.split('/')[-2]}-readme.txt" in txts or f"{GLOBAL_PATH}{path}{path.split('/')[-2]}-README.txt" in txts:
            matrix["readme"].append(True)
        else:
            matrix["readme"].append(False)
        
        if not txtFound:
            for txt in txts:
                if txt.lower().split("/")[-1] not in [f"{id}.txt", f"{id}-8.txt", f"{id}-0.txt", f"{id}-readme.txt"]:
                    raise Exception(f"New file format found: {txt}")
            files_missing.append(id)
    return matrix, files_missing



In [11]:
#ids = list(range(1, 100000))
#exceptions = [4682, 4769, 10681, 12884, 16529, 71264] # there is a naming issue, needs manual maintenance
#missing = ["928", "1630", "8746", "9933", "10547", "10634", "13080", "13447", "13609", "13692", "13693", "13702", "14550", "14624",
#          "15114", "17204"]
#broken = ["40"]
#[ids.remove(ex) for ex in missing]
#[ids.remove(ex) for ex in broken]
#build_encoding_matrix(["10681"])
formats, files_missing = build_encoding_matrix(ids)
print(len(files_missing))
print(files_missing)

df_formats = pd.DataFrame.from_dict(formats)

# Filter rows where all three columns are False
#filtered_df = df_formats[(df_formats['ascii'] == False) & (df_formats['utf'] == False) & (df_formats['ISO-8859-1'] == False)]

# Count the number of such rows
#count_false_rows = len(filtered_df)
#print(count_false_rows)
df_formats.info()

mask = ~df_formats['ids'].isin(files_missing)
mask_tmp = ~tmp['id'].isin(files_missing)

# Apply the mask to the DataFrame
df_formats = df_formats[mask]
tmp = tmp[mask_tmp]
tmp.info()


75
['928', '8746', '9933', '10547', '10634', '13447', '13609', '13692', '13693', '13702', '14550', '14624', '15114', '17204', '17224', '17576', '18741', '19770', '20073', '25155', '25222', '25564', '26373', '26839', '29779', '29782', '29785', '29788', '29914', '31061', '31076', '31246', '32000', '32060', '32497', '32625', '32857', '33063', '33283', '36154', '36525', '36670', '36884', '37030', '37275', '38079', '38640', '38722', '38769', '38993', '39041', '40395', '40624', '41568', '41654', '47464', '50977', '50992', '57355', '59248', '68960', '69374', '69512', '69513', '69831', '69832', '70888', '71026', '71291', '71675', '72787', '72875', '72963', '73297', '77279']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25136 entries, 0 to 25135
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ids         25136 non-null  object
 1   ascii       25136 non-null  bool  
 2   utf         25136 non-null  bool  
 3   ISO-8859-1  

In [12]:
tmp.to_csv('./GutenChrono_catalogue.csv', index=False)

In [131]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        confidence = result['confidence']
        return encoding, confidence

**Processing**
Borrows from

- https://github.com/pgcorpus/gutenberg/blob/master/src/pipeline.py

- https://github.com/c-w/gutenberg/blob/master/gutenberg/cleanup/strip_headers.py

In [138]:
TEXT_START_MARKERS = frozenset((
    "*END*THE SMALL PRINT",
    "*** START OF THE PROJECT GUTENBERG",
    "*** START OF THIS PROJECT GUTENBERG",
    "This etext was prepared by",
    "E-text prepared by",
    "Produced by",
    "Distributed Proofreading Team",
    "Proofreading Team at http://www.pgdp.net",
    "http://gallica.bnf.fr)",
    "      http://archive.org/details/",
    "http://www.pgdp.net",
    "by The Internet Archive)",
    "by The Internet Archive/Canadian Libraries",
    "by The Internet Archive/American Libraries",
    "public domain material from the Internet Archive",
    "Internet Archive)",
    "Internet Archive/Canadian Libraries",
    "Internet Archive/American Libraries",
    "material from the Google Print project",
    "*END THE SMALL PRINT",
    "***START OF THE PROJECT GUTENBERG",
    "This etext was produced by",
    "*** START OF THE COPYRIGHTED",
    "The Project Gutenberg",
    "http://gutenberg.spiegel.de/ erreichbar.",
    "Project Runeberg publishes",
    "Beginning of this Project Gutenberg",
    "Project Gutenberg Online Distributed",
    "Gutenberg Online Distributed",
    "the Project Gutenberg Online Distributed",
    "Project Gutenberg TEI",
    "This eBook was prepared by",
    "http://gutenberg2000.de erreichbar.",
    "This Etext was prepared by",
    "This Project Gutenberg Etext was prepared by",
    "Gutenberg Distributed Proofreaders",
    "Project Gutenberg Distributed Proofreaders",
    "the Project Gutenberg Online Distributed Proofreading Team",
    "**The Project Gutenberg",
    "*SMALL PRINT!",
    "More information about this book is at the top of this file.",
    "tells you about restrictions in how the file may be used.",
    "l'authorization à les utilizer pour preparer ce texte.",
    "of the etext through OCR.",
    "*****These eBooks Were Prepared By Thousands of Volunteers!*****",
    "We need your donations more than ever!",
    " *** START OF THIS PROJECT GUTENBERG",
    "****     SMALL PRINT!",
    '["Small Print" V.',
    '      (http://www.ibiblio.org/gutenberg/',
    'and the Project Gutenberg Online Distributed Proofreading Team',
    'Mary Meehan, and the Project Gutenberg Online Distributed Proofreading',
    '                this Project Gutenberg edition.',
))


TEXT_END_MARKERS = frozenset((
    "*** END OF THE PROJECT GUTENBERG",
    "*** END OF THIS PROJECT GUTENBERG",
    "***END OF THE PROJECT GUTENBERG",
    "End of the Project Gutenberg",
    "End of The Project Gutenberg",
    "Ende dieses Project Gutenberg",
    "by Project Gutenberg",
    "End of Project Gutenberg",
    "End of this Project Gutenberg",
    "Ende dieses Projekt Gutenberg",
    "        ***END OF THE PROJECT GUTENBERG",
    "*** END OF THE COPYRIGHTED",
    "End of this is COPYRIGHTED",
    "Ende dieses Etextes ",
    "Ende dieses Project Gutenber",
    "Ende diese Project Gutenberg",
    "**This is a COPYRIGHTED Project Gutenberg Etext, Details Above**",
    "Fin de Project Gutenberg",
    "The Project Gutenberg Etext of ",
    "Ce document fut presente en lecture",
    "Ce document fut présenté en lecture",
    "More information about this book is at the top of this file.",
    "We need your donations more than ever!",
    "END OF PROJECT GUTENBERG",
    " End of the Project Gutenberg",
    " *** END OF THIS PROJECT GUTENBERG",
))

LEGALESE_START_MARKERS = frozenset(("<<THIS ELECTRONIC VERSION OF",))
LEGALESE_END_MARKERS = frozenset(("SERVICE THAT CHARGES FOR DOWNLOAD",))

def strip_headers(text):
    """
    Remove lines that are part of the Project Gutenberg header or footer.

    Note: this function is a port of the C++ utility by Johannes Krugel. The
    original version of the code can be found at:
    http://www14.in.tum.de/spp1307/src/strip_headers.cpp

    Args:
        text (unicode): The body of the text to clean up.

    Returns:
        unicode: The text with any non-text content removed.

    """
    lines = text.splitlines()
    sep = str(os.linesep)

    out = []
    i = 0
    footer_found = False
    ignore_section = False

    for line in lines:
        reset = False

        if i <= 600:
            # Check if the header ends here
            if any(line.startswith(token) for token in TEXT_START_MARKERS):
                reset = True

            # If it's the end of the header, delete the output produced so far.
            # May be done several times, if multiple lines occur indicating the
            # end of the header
            if reset:
                out = []
                continue

        if i >= 100:
            # Check if the footer begins here
            if any(line.startswith(token) for token in TEXT_END_MARKERS):
                footer_found = True

            # If it's the beginning of the footer, stop output
            if footer_found:
                break

        if any(line.startswith(token) for token in LEGALESE_START_MARKERS):
            ignore_section = True
            continue
        elif any(line.startswith(token) for token in LEGALESE_END_MARKERS):
            ignore_section = False
            continue

        if not ignore_section:
            out.append(line.rstrip(sep))
            i += 1

    return sep.join(out)

def process_book(path_to_raw_file, encoding):
        # read raw file
        try:
            if encoding == "utf-8": 
                with io.open(path_to_raw_file, encoding="UTF-8") as f:
                    text = f.read()
            elif encoding == "ascii":
                # Encoding is not always ascii
                with io.open(path_to_raw_file, encoding="ascii") as f:
                    text = f.read()

            elif encoding == "ISO-8859-1": 
                with io.open(path_to_raw_file, encoding='iso-8859-1') as f:
                    text = f.read()
            else:
                raise Exception("Unknown Encoding")
        except:
            encoding, confidence = detect_encoding(path_to_raw_file)
            if confidence >= 0.5:
                with io.open(path_to_raw_file, encoding=encoding, errors='replace') as f:
                    text = f.read()
            else: 
                raise Exception("too low confidence")  
                
        # clean it up
        clean = strip_headers(text)
        return clean

In [139]:
def build_yearly_chunk(ids, encoding_matrix):
    text = ""
    for id in ids:
        path = build_file_path(id)
        if encoding_matrix.loc[encoding_matrix["ids"] == str(id), "utf"].values[0]:
            unicode_text = process_book(GLOBAL_PATH+path+str(id)+"-8.txt", "utf-8")
            text += unicode_text
            
        # check for ascii txt first as it can be converted to UTF-8 easily
        elif encoding_matrix.loc[encoding_matrix["ids"] == str(id), "ascii"].values[0]:
            unicode_text = process_book(GLOBAL_PATH+path+str(id)+".txt", "ascii")
            text += unicode_text
            
        elif encoding_matrix.loc[encoding_matrix["ids"] == str(id), "ISO-8859-1"].values[0]:
            unicode_text = process_book(GLOBAL_PATH+path+str(id)+"-0.txt", "ISO-8859-1")
            text += unicode_text
        
        else:
            raise Exception(f"No text found for id: {id}")
    return text
        
    

In [140]:
tmp.loc[:, "year_majority_vote"] = tmp.apply(
    lambda row: int(float(row["year_majority_vote"])) if not pd.isna(row["year_majority_vote"]) else row["year_majority_vote"], 
    axis=1
)
year_to_ids = {}
ids = tmp['id'].tolist()
years = tmp['year_majority_vote'].tolist()
for id_, year in zip(ids, years):
    year = year
    if year not in year_to_ids:
        year_to_ids[year] = []
    year_to_ids[year].append(id_)

In [141]:
print(year_to_ids)

{1776: ['1', '147', '300', '731', '732', '733', '734', '735', '736', '890', '891', '892', '893', '894', '895', '1564', '2092', '2093', '2094', '3300', '3755', '6731', '9180', '10357', '16780', '21002', '22387', '29226', '38194', '69351'], 1961: ['3', '18137', '18172', '19090', '19478', '19726', '20154', '20289', '23624', '25067', '25544', '26038', '29285', '29448', '29563', '30371', '31035', '31334', '31429', '31513', '32482', '32903', '33509', '33543', '33966', '34787', '37740', '42901', '43884', '50822', '52650', '55168', '57010', '58136', '58803', '58912', '59398', '60070', '66346', '66843', '67199', '68105'], 1863: ['4', '1018', '1416', '2291', '2301', '2303', '2440', '2522', '2659', '2921', '2922', '2923', '2924', '2925', '2926', '2931', '3327', '3837', '4536', '4537', '4925', '4927', '6335', '6962', '8090', '8110', '8163', '8699', '10325', '11224', '12470', '15672', '15689', '16493', '16581', '17002', '17612', '17769', '17826', '18135', '19705', '19875', '19999', '20928', '21708'

In [142]:
# Specify the file path where you want to save the string
file_path = "./txts/"
yearly_txts = {}
for key in year_to_ids.keys():
    txt = build_yearly_chunk(year_to_ids[key], df_formats)
    yearly_txts[key] = txt
    # Open the file in write mode with UTF-8 encoding and write the string to it
    with open(file_path+f"{key}.txt", 'w', encoding='utf-8') as file:
        file.write(txt)
    print(f"Successfully saved txt for year: {key}")

    

Successfully saved txt for year: 1776
Successfully saved txt for year: 1961
Successfully saved txt for year: 1863
Successfully saved txt for year: 1865
Successfully saved txt for year: 1871
Successfully saved txt for year: 1876
Successfully saved txt for year: 1851
Successfully saved txt for year: 1906
Successfully saved txt for year: 1830
Successfully saved txt for year: 1787
Successfully saved txt for year: 1855
Successfully saved txt for year: 1805
Successfully saved txt for year: 1845
Successfully saved txt for year: 1913
Successfully saved txt for year: 1874
Successfully saved txt for year: 1915
Successfully saved txt for year: 1850
Successfully saved txt for year: 1895
Successfully saved txt for year: 1897
Successfully saved txt for year: 1820
Successfully saved txt for year: 1875
Successfully saved txt for year: 1908
Successfully saved txt for year: 1843
Successfully saved txt for year: 1909
Successfully saved txt for year: 1920
Successfully saved txt for year: 1904
Successfully

Successfully saved txt for year: 1972
Successfully saved txt for year: 1978
Successfully saved txt for year: 1968
Successfully saved txt for year: 1985
Successfully saved txt for year: 1981
Successfully saved txt for year: 1995
Successfully saved txt for year: 1973
Successfully saved txt for year: 1997
Successfully saved txt for year: 1976
Successfully saved txt for year: 1982
Successfully saved txt for year: 1977
Successfully saved txt for year: 1970
Successfully saved txt for year: 1966
Successfully saved txt for year: 1971
Successfully saved txt for year: 1986
Successfully saved txt for year: 1984
Successfully saved txt for year: 1780
Successfully saved txt for year: 1967
Successfully saved txt for year: 1769
Successfully saved txt for year: 1758
Successfully saved txt for year: 1763
Successfully saved txt for year: 1987
Successfully saved txt for year: 1994
Successfully saved txt for year: 1999
Successfully saved txt for year: 1992
Successfully saved txt for year: 2000
Successfully

In [ ]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        confidence = result['confidence']
        return encoding, confidence

    
    
# Example usage:
file_path = './mirror/6/7/3/6731/6731.txt'
encoding, confidence = detect_encoding(file_path)

print(f'The detected encoding is {encoding} with {confidence * 100:.2f}% confidence.')

In [ ]:
tmp.to_csv('./pg_catalog_filtered.csv', index=False)